In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

import re

In [2]:
# patients = pd.read_csv('../data/pts.csv')

# patients.columns = [x.lower().replace(' ', '_') for x in patients.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# patients.to_sql('patients', db, if_exists = 'append', index = False) 

299712

In [3]:
# admission = pd.read_csv('../data/hos/admissions.csv')

# admission.columns = [x.lower().replace(' ', '_') for x in admission.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# admission.to_sql('admission', db, if_exists = 'append', index = False) 

431231

In [4]:
# diagicd = pd.read_csv('../data/diag_icd.csv')

# diagicd.columns = [x.lower().replace(' ', '_') for x in diagicd.columns]

# db = sqlite3.connect('../data/ehr.sqlite')

# diagicd.to_sql('diagicd', db, if_exists = 'append', index = False) 

4756326

In [ ]:
# subject_id	hadm_id patient

In [31]:
dis.loc[j, 'text']

' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n \nChief Complaint:\nDyspnea and chest tightness\n \nMajor Surgical or Invasive Procedure:\n___ Pharmacologic nuclear stress test\n \nHistory of Present Illness:\n___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib c

In [103]:
text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']


tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])



# for i in range (len(text_subheading)-1):
#     text_parsed =tt1.pop()

# print(i, text_parsed, len('text_subheading'))

In [104]:
tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
dic = {}
tt1.pop(0)

while len(tt1) > 0:
    
    key = tt1.pop(0)
    value = tt1.pop(0)

    if key in text_subheading:
        dic[key] = value

In [105]:
dic

{'Chief Complaint': 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure': '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness': '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and across lower \nrib cage, persistent since onset, no radiation to the \nshoulders/jaw/back, no diaphoreses. Worsens with activity, \nimproves somewhat with rest. Patient does not it feels like \nother episodes when she then required her stent placement. Took \n

In [95]:
tt1

[' \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___.\n ',
 'Chief Complaint',
 'Dyspnea and chest tightness\n ',
 'Major Surgical or Invasive Procedure',
 '___ Pharmacologic nuclear stress test\n ',
 'History of Present Illness',
 '___ year old female with history of HTN, HLD, hx of CVA, CAD s/p \nBMS to circumflex and POBA (___), on Aspirin and Plavix, CHF \n(EF 45% in ___, diabetes, presenting with acute onset \nshortness of breath and substernal chest tightness since ___ \nevening. \n\nPatient notes that ___ evening had a large seafood dinner \nwhich is not usual for her, and then later around 10pm had acute \nonset of SOB feeling like she could not take deep breaths with \nchest tightness (patient notes this is her "angina"). Denies \npleuritic component to CP, described as central and acro

In [76]:
# tt1.index('Discharge Instructions')
# i = 'Chief Complaint'
for i in range (len(list_subheading)-1):
#     index = tt1.index(i)
    text_parsed =tt1.pop(i+1)
    
    text_parsed

In [116]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_e78 = pd.read_csv('../data/icd/e78.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_e78, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [119]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_i16 = pd.read_csv('../data/icd/i16.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_i16, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [121]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_y92 = pd.read_csv('../data/icd/y92.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_y92, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [123]:
# db = sqlite3.connect('../data/ehr.sqlite')

# text_subheading = ['Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions', '$']

# df_e11 = pd.read_csv('../data/icd/e11.csv')
# for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
#     dis = chunk.merge(df_e11, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
#     dic_list = []
#     for j in range (len(dis)):
#         tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
#         dic = {'subject_id': (dis.loc[j, 'subject_id']),
#                'hadm_id' : (dis.loc[j, 'hadm_id']),
#               'icd' : (dis.loc[j, 'icd']),
#               'long_title' : (dis.loc[j, 'long_title'])}
#         tt1.pop(0)

#         while len(tt1) > 0:
    
#             key = tt1.pop(0)
#             value = tt1.pop(0)

#             if key in text_subheading:
#                 dic[key] = value

      
 
#         dic_list.append(dic)


#     df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
#      'Chief Complaint',
#      'Major Surgical or Invasive Procedure',
#      'History of Present Illness',
#      'Past Medical History',
#      'Social History',
#      'Family History',
#      'Physical Exam',
#      'Discharge',
#      'Pertinent Results',
#      'Brief Hospital Course',
#      'Medications on Admission',
#      'Discharge Medications',
#      'Discharge Disposition',
#      'Discharge Diagnosis',
#      'Discharge Condition',
#      'Discharge Instructions',
#      'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
#      'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
#      'History of Present Illness': 'history_present_illness',
#      'Past Medical History': 'past_medcal_history',
#      'Social History':'social_history',
#      'Family History':'family_history',
#      'Physical Exam':'physical_exam',
#      'Discharge':'discharge',
#      'Pertinent Results': 'pertinent_results',
#      'Brief Hospital Course': 'hospital_course',
#      'Medications on Admission':'medication_on_admission',
#      'Discharge Medications':'medication_on_discharge',
#      'Discharge Disposition': 'discharge_disposition',
#      'Discharge Diagnosis': 'discharge_diagnosis',
#      'Discharge Condition':'discharge_condition',
#      'Discharge Instructions':'discharge_instruction',
#      'Followup Instructions':'followup_instruction'})
   


#         # filter it to the subjectId and hadm_id
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

0it [00:00, ?it/s]

In [ ]:
df_text.to_sql('text', db, if_exists = 'append', index = False)

In [12]:
df_text

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,18442315,26150389,I50,"Congestive heart failure, unspecified",shortness of breath \n,None. \n\n,___ y/o female with a past medical history of ...,PMH: \n1. Hypertension.\n2. Osteoporosis.\n...,___,non-contributory\n\n,...,NaN,ADMISSION LABS:\n================\n___ 01:02PM...,___ y/o female with a past medical history of ...,The Preadmission Medication list is accurate a...,1. Rivaroxaban 15 mg PO DAILY \nTake with the ...,Extended Care\n,Atrial Fibrillation\nCOPD exacerbation\nUrinar...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt has been a pleasure taking...",___\n
1,18442315,26150389,I50,Acute systolic heart failure,shortness of breath \n,None. \n\n,___ y/o female with a past medical history of ...,PMH: \n1. Hypertension.\n2. Osteoporosis.\n...,___,non-contributory\n\n,...,NaN,ADMISSION LABS:\n================\n___ 01:02PM...,___ y/o female with a past medical history of ...,The Preadmission Medication list is accurate a...,1. Rivaroxaban 15 mg PO DAILY \nTake with the ...,Extended Care\n,Atrial Fibrillation\nCOPD exacerbation\nUrinar...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt has been a pleasure taking...",___\n
2,18442432,27855871,I50,Unspecified diastolic (congestive) heart failure,Bright red blood per rectum\n,None\n,This is a ___ woman with a history of Afib s/p...,___ esophagus \nS/P Mechanical aortic valve \n...,___,Her father died at age ___- MI. ___ mother die...,...,NaN,ADMISSION LABS\n___ 05:05PM BLOOD WBC-7.7 RBC-...,NaN,The Preadmission Medication list is accurate a...,1. Enoxaparin Sodium 70 mg SC Q12H \nStart: T...,Home\n,Lower gastrointestinal bleeding\nColon polyps\...,Mental Status: Clear and coherent.\nLevel of C...,You were hospitalized for bleeding after colon...,___\n
3,18443495,24686393,I50,Unspecified diastolic (congestive) heart failure,Seizures\n,intubation\n,Eu Critical ___ aka ___ (MRN ___ is a ___ with...,# Type 1 diabetes diagnosed at age ___. H/o me...,___,Her father has AIDS. Her mother has diabetes a...,...,NaN,===ADMISSION LABS===\n___ 04:19PM BLOOD WBC-13...,"#SEIZURE\nAdmitted to ICU, intubated/sedated f...",The Preadmission Medication list is accurate a...,1. Divalproex (DELayed Release) 1000 mg PO Q8...,Home With Service\n,status epilepticus\nend stage renal disease\n\n,Mental Status: Clear and coherent.\nLevel of C...,"Ms. ___, \nYou were admitted with seizures. To...",___\n
4,18443821,20382977,I50,Acute on chronic diastolic (congestive) heart ...,"Dyspnea, leg swelling, chest pain\n",none\n\n,___ with history of CAD (s/p 1x stent in ___ i...,- CAD s/p 1x stent (placed in ___ in ___ \n- ...,___,___ family history of early MI or history of h...,...,NaN,ADMISSION LABS:\n==============\n___ 09:46PM B...,"___ with CAD s/p PCI ___ (unknown details), CK...",Patient does not know his medications. On adm...,1. amLODIPine 10 mg PO DAILY \nRX *amlodipine...,Home\n,Primary Diagnosis:\nAcute exacerbation of dias...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Mr. ___,\n\nYou were admitted to the hosp...",___\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2422,18731959,21511199,I50,Acute on chronic systolic (congestive) heart f...,Elective admission for percutaneous coronary i...,___ Cardiac catheterization \n,___ year old woman with of lung CA (small cell...,1. Lung cancer diagnosed ___: s/p chemotherapy...,___,"Brother alive and well, age ___. Mother health...",...,NaN,===============,Ms. ___ is a ___ with a background history of ...,1. Anastrozole 1 mg PO DAILY \n2. Aspirin 81 m...,1. Azithromycin 250 mg PO DAILY Duration: 4 D...,Home\n,

In [21]:
#checking the tables in the sqlite_db

db = sqlite3.connect('../data/ehr.sqlite')
db.execute("""SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';""").fetchall()

[('patients',), ('admission',), ('diagicd',), ('text',), ('text_ic50',)]

In [22]:
# query = """
# SELECT * 
# FROM text 
# WHERE subject_id = '10000980'
# """
query = """
SELECT COUNT (*) 
FROM text
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,COUNT (*)
0,0


In [11]:
query = """
SELECT * 
FROM text 
WHERE icd = 'I50'
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction
0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,Denies cardiac family history. Family hx of DM...,...,None,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
1,10000980,29654838,I50,Acute on chronic diastolic heart failure,Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,Denies cardiac family history. Family hx of DM...,...,None,Admission Labs\n___ 01:18PM BLOOD WBC-6.4# RBC...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n
2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,"Niece had some sort of cancer. Otherwise, no f...",...,None,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
3,10000980,26913865,I50,"Congestive heart failure, unspecified",dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,"Niece had some sort of cancer. Otherwise, no f...",...,None,ON ADMISSION:\n\n___ 06:48AM BLOOD WBC-5.3 RBC...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n
4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,Niece had some sort of cancer. Father died in ...,...,None,==== ADMISSION LABS ====\n___ 04:00PM BLOOD WB...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68489,18439835,22458701,I50,Chronic diastolic heart failure,Pt presenting for initiation of dialysis\n,None\n\n,Mrs. ___ is a pleasant ___ yo F w/ stage V CKD...,- Type 2 Diabetes \n- Coronary artery disease...,___,Mother with heart disease in ___ and cervical ...,...,None,On admission\n___ 02

In [5]:
#As the database crashed in the middle of adding IC50 rows we wanted to delete and restore the table 
#so that when adding baack there is no duplication of rows.
# query1 = """
# DELETE FROM text 
# WHERE icd = 'I50';
# """

# query2 = """
# COMMIT;
# """

#maximum allowed pages n db
query3= """
PRAGMA max_page_count;  
"""

# number of pages we have in db
query4= """
PRAGMA page_count;
"""

query5= """
PRAGMA temp_store_directory;
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
#    page_count = db.execute(query4)
    directory = db.execute(query5)
# page_count.fetchall()
directory.fetchall()


#1073741823 max_count, page#we have (1046870,)

[]

In [9]:
query = """
SELECT * 
FROM text 
WHERE icd = 'I50'
"""

with sqlite3.connect('../data/ehr.sqlite') as db: 
    text_sqlite = pd.read_sql(query, db)
text_sqlite

,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,family_history,...,discharge,pertinent_results,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction


In [5]:
# db = sqlite3.connect('../data/ehr.sqlite')

df = []

text_subheading = ['Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions', '$']

# df_i50 = pd.read_csv('../data/icd/i50.csv')
df_e11 = pd.read_csv('../data/icd/e11.csv')
for chunk in tqdm(pd.read_csv('../data/text/discharge.csv', chunksize = 10000)):
    
    dis = chunk.merge(df_e11, how='inner', on=['subject_id', 'hadm_id']) #to get the rows that match our observations
    
    
    
    dic_list = []
    for j in range (len(dis)):
        tt1 = re.split('\n(.+):\n', dis.loc[j,'text'])
        dic = {'subject_id': (dis.loc[j, 'subject_id']),
               'hadm_id' : (dis.loc[j, 'hadm_id']),
              'icd' : (dis.loc[j, 'icd']),
              'long_title' : (dis.loc[j, 'long_title'])}
        tt1.pop(0)

        while len(tt1) > 0:
    
            key = tt1.pop(0)
            value = tt1.pop(0)

            if key in text_subheading:
                dic[key] = value

      
 
        dic_list.append(dic)


    df_text = pd.DataFrame(dic_list) [['subject_id', 'hadm_id', 'icd', 'long_title',
     'Chief Complaint',
     'Major Surgical or Invasive Procedure',
     'History of Present Illness',
     'Past Medical History',
     'Social History',
     'Family History',
     'Physical Exam',
     'Discharge',
     'Pertinent Results',
     'Brief Hospital Course',
     'Medications on Admission',
     'Discharge Medications',
     'Discharge Disposition',
     'Discharge Diagnosis',
     'Discharge Condition',
     'Discharge Instructions',
     'Followup Instructions']].rename(columns = {'Chief Complaint': 'chief_complaint',
     'Major Surgical or Invasive Procedure': 'surgical_invasive_procedure',
     'History of Present Illness': 'history_present_illness',
     'Past Medical History': 'past_medcal_history',
     'Social History':'social_history',
     'Family History':'family_history',
     'Physical Exam':'physical_exam',
     'Discharge':'discharge',
     'Pertinent Results': 'pertinent_results',
     'Brief Hospital Course': 'hospital_course',
     'Medications on Admission':'medication_on_admission',
     'Discharge Medications':'medication_on_discharge',
     'Discharge Disposition': 'discharge_disposition',
     'Discharge Diagnosis': 'discharge_diagnosis',
     'Discharge Condition':'discharge_condition',
     'Discharge Instructions':'discharge_instruction',
     'Followup Instructions':'followup_instruction'})
   


        # filter it to the subjectId and hadm_id
    df.append(df_text)
df_icd = pd.concat(df)
df_icd.to_csv('../data/text_ice11.csv') #add .csv
#     df_text.to_sql('text', db, if_exists = 'append', index = False)            # Append the chunk to a referral table

    

0it [00:00, ?it/s]

In [4]:
text_ic50=pd.read_csv("../data/text_ic50")

In [7]:
text_e11=pd.read_csv("../data/text_ice11")

In [10]:
text_ic50.shape #(80909, 22)
text_e11.shape #(108625, 22)

(108625, 22)

In [12]:
dftexti50_e11 = pd.concat([text_ic50, text_e11])

In [14]:
dftexti50_e11.shape #(189534, 22)
dftexti50_e11.columns

Index(['Unnamed: 0', 'subject_id', 'hadm_id', 'icd', 'long_title',
       'chief_complaint', 'surgical_invasive_procedure',
       'history_present_illness', 'past_medcal_history', 'social_history',
       'family_history', 'physical_exam', 'discharge', 'pertinent_results',
       'hospital_course', 'medication_on_admission', 'medication_on_discharge',
       'discharge_disposition', 'discharge_diagnosis', 'discharge_condition',
       'discharge_instruction', 'followup_instruction'],
      dtype='object')

In [15]:
df_pivot = dftexti50_e11[['subject_id', 'hadm_id', 'icd']]

In [34]:
dftexti50_e11.drop_duplicates(['subject_id', 'hadm_id']).merge(target1, how='inner', on=['subject_id', 'hadm_id'])

,Unnamed: 0,subject_id,hadm_id,icd,long_title,chief_complaint,surgical_invasive_procedure,history_present_illness,past_medcal_history,social_history,...,hospital_course,medication_on_admission,medication_on_discharge,discharge_disposition,discharge_diagnosis,discharge_condition,discharge_instruction,followup_instruction,icd_E11,icd_I50
0,0,10000980,29654838,I50,"Congestive heart failure, unspecified",Shortness of breath\n,None\n,"___ yo woman with h/o hypertension, hyperlipid...","1. CAD RISK FACTORS: +Diabetes, +Dyslipidemia,...",___,...,"___ woman with h/o hypertension, hypelipidemia...",ATENOLOL - 100 mg Tablet - 1.5 Tablet(s) by mo...,1. clopidogrel 75 mg Tablet Sig: One (1) Table...,Home With Service\n,-Hypertension with hypertensive urgency\n-Myoc...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___, \n\nYou were admitted for shortn...",___\n,1,1
1,2,10000980,26913865,I50,Acute on chronic systolic heart failure,dyspnea\n,Cardiac catheterization ___\n\n,"This is a ___ M with history of diabetes, dias...",- hypertension\n- diabetes\n- hx CVA (cerebell...,___,...,"___ female with ___, HTN, diabetes, CKD presen...",The Preadmission Medication list is accurate a...,1. Aspirin 81 mg PO DAILY \n2. Clopidogrel 75 ...,Home With Service\n,PRIMARY DIAGNOSIS:\nSevere mitral regurgitatio...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n,1,1
2,4,10000980,24947999,I50,Acute on chronic systolic heart failure,Dyspnea and chest tightness\n,___ Pharmacologic nuclear stress test\n,"___ year old female with history of HTN, HLD, ...",- hypertension \n- diabetes \n- hx CVA (cere...,___,...,"___ year old female with history of HTN, CVA, ...",The Preadmission Medication list is accurate a...,1. Outpatient Lab Work\nPlease draw Chem-10 an...,Home With Service\n,Primary Diagnosis:\n- Acute on chronic decompe...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\n___ were admitted to ___ on ___ w...",___\n,1,1
3,6,10000980,25242409,I50,"Congestive heart failure, unspecified",DVT\n,EGD\n\n,This is a ___ y/o female with PMHx significant...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,Outpatient Providers: ___ with PMHx significan...,The Preadmission Medication list is accurate a...,1. Allopurinol ___ mg PO EVERY OTHER DAY \n2. ...,Extended Care\n,Primary:\nDeep vein thrombosis (DVT)\n,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you ...",___\n,1,1
4,8,10000980,25911675,I50,Acute on chronic systolic (congestive) heart f...,"fatigue, anemia\n",None\n\n,Ms. ___ is a ___ year old woman with a past me...,- hypertension \n- diabetes \n- hx CVA (cere...,___,...,Ms. ___ is a ___ year old woman with a past me...,The Preadmission Medication list may be inaccu...,1. Acetaminophen 325-650 mg PO Q6H:PRN pain or...,Home With Service\n,Primary diagnosis:\nAnemia\nCongestive heart f...,Mental Status: Clear and coherent.\nLevel of C...,"Dear ___,\n\nIt was a pleasure caring for you....",___\n,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114683,556,19999287,22997012,E11,Diabetes mellitus without mention of complicat...,Shortness of breath\n,___\n1. Flexible and rigid bronchoscopy\n2. Cr...,"Ms. ___ is a ___ with a history of COPD, T1 N0...","COPD, Pulmonary function tests on ___ showed a...",___,...,Ms. ___ is a ___ with a history of poorly diff...,The Preadmission Medication list is accurate a...,1. Amlodipine 5 mg PO DAILY \n2. Bisacodyl 5 m...,Home With Service\n,Primary: \nPost-obstructive pneumonia\nRecurre...,Mental Status: Clear and coherent.\nLevel of C...,"Dear Ms. ___,\n\nIt was a pleasure caring for ...",___\n,1,0
114684,557,19999287,20175828,E11,Diabetes mellitus without mention of complicat...,dyspnea\n,___ - Bronchoscopy\n\n,___ hx COPD (FEV1 49% predicted ___ and poorly...,COPD Pulmonary function tests on\n___ showed a...,___,...,This is a ___ year old female with

In [17]:
df_pivot.tail()

,subject_id,hadm_id,icd
108620,19999287,22997012,E11
108621,19999287,20175828,E11
108622,19999379,26008899,E11
108623,19999828,29734428,E11
108624,19999828,25744818,E11


In [29]:
target1 = ((pd.get_dummies(df_pivot, columns = ['icd']).groupby(['subject_id', 'hadm_id'])
            .sum()>0).astype(int).reset_index())

In [30]:
target1

,subject_id,hadm_id,icd_E11,icd_I50
0,10000980,20897796,1,1
1,10000980,24947999,1,1
2,10000980,25242409,1,1
3,10000980,25911675,1,1
4,10000980,26913865,1,1
...,...,...,...,...
114683,19999287,20175828,1,0
114684,19999287,22997012,1,0
114685,19999379,26008899,1,0
114686,19999828,25744818,1,0


In [21]:
df_pivot.pivot_table(columns='icd', index = ['subject_id', 'hadm_id'])

Empty DataFrame
Columns: []
Index: [(10000980, 20897796), (10000980, 24947999), (10000980, 25242409), (10000980, 25911675), (10000980, 26913865), (10000980, 29654838), (10000980, 29659838), (10001176, 23334588), (10001667, 22672901), (10001877, 21320596), (10001877, 25679292), (10002013, 21763296), (10002013, 21975601), (10002013, 23581541), (10002013, 24760295), (10002013, 24848509), (10002013, 25442395), (10002013, 25715803), (10002013, 27574273), (10002013, 28420602), (10002013, 28603984), (10002131, 24065018), (10002155, 23822395), (10002155, 28994087), (10002221, 20195471), (10002221, 20237862), (10002221, 21729093), (10002221, 29399017), (10002428, 23473524), (10002430, 24513842), (10002430, 24648311), (10002430, 26295318), (10002430, 27218502), (10002443, 21329021), (10002495, 24982426), (10002760, 28094813), (10002976, 21640325), (10002976, 27179825), (10003019, 28836362), (10003299, 20940957), (10003299, 21476780), (10003299, 29323205), (10003400, 20214994), (10003400, 23559586), (10003400, 26467376), (10003502, 20459702), (10003502, 26475031), (10003502, 29011269), (10003637, 22082422), (10003637, 23487925), (10003637, 28484061), (10004235, 24181354), (10004322, 20356134), (10004322, 22602599), (10004322, 28755331), (10004401, 21085166), (10004401, 22869003), (10004401, 23920883), (10004401, 25753439), (10004401, 25777141), (10004401, 26164209), (10004401, 26488315), (10004401, 27939719), (10004401, 28128182), (10004401, 28154772), (10004401, 28679787), (10004401, 29988601), (10005012, 28371912), (10005236, 23384508), (10005236, 25656545), (10005368, 24631981), (10005749, 20010003), (10005749, 24015009), (10005749, 24979738), (10005817, 20626031), (10005817, 28661809), (10005858, 29569314), (10006029, 20850584), (10006029, 25426298), (10006029, 27104518), (10006029, 29107716), (10006277, 25610553), (10006457, 24934308), (10006457, 27894366), (10006513, 28504108), (10006513, 29846618), (10006580, 24159665), (10006640, 29151310), (10007174, 20280072), (10007174, 22934294), (10007462, 29383640), (10007795, 20285402), (10007795, 27962747), (10007818, 22987108), (10009614, 27624592), (10009628, 25926192), (10010058, 21955805), (10010058, 25917765), (10010058, 26359957), (10010058, 28963312), ...]

[114688 rows x 0 columns]